<a href="https://colab.research.google.com/github/akinzayn/NMSUCovidPrevalenceProject/blob/main/Qualtrics_One_drive_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setting Up Colab**

In [ ]:
#@title
install.packages("pacman") 
library(pacman) # package for computing sensitivity and specificity

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [209]:
#@title
h = p_load(tidyverse, googledrive, readxl, qualtRics, openxlsx, Microsoft365R)


In [208]:
drive_deauth()
integrity_files = drive_get(as_id(c("")))
h = 1:dim(integrity_files)[1] %>% map(~drive_download(integrity_files[.,], overwrite = TRUE))

File downloaded:

• Covid_Integrity_Apr_23_5_09_Am
  <id: 1f6SgUU6wbPAX1Yz8nK9i8ZmxfkQ8D5kb1qxFCBTok4E>

Saved locally as:

• Covid_Integrity_Apr_23_5_09_Am.xlsx



##### **Downloading Data From Qualtrics**

In [ ]:
qualtrics_api_credentials(api_key = "", 
                          base_url = "sjc1.qualtrics.com",
                          install = TRUE, overwrite = TRUE)

Your Qualtrics key and base URL have been stored in your .Renviron.  
To use now, restart R or run `readRenviron("~/.Renviron")`



In [ ]:
readRenviron("~/.Renviron")

In [ ]:
surveys <- all_surveys() 

In [ ]:
head(surveys, 2)

id,name,ownerId,lastModified,creationDate,isActive
<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
SV_24YyVO3ywHx3Agu,Prevalence of Depression & Sleep Disorders in New Mexico State University Population during COVID-19,UR_0uZUwCWAtCSaKAC,2022-04-22T23:06:28Z,2022-04-01T05:21:24Z,TRUE


In [207]:

survey_response_df <- fetch_survey(surveyID = surveys$id[1], 
                         verbose = TRUE,
                        #  strip_html = TRUE,
                          import_id = TRUE,
                           time_zone = "America/Denver",
                            add_column_map = TRUE,
                             add_var_labels = TRUE,
                             force_request=TRUE, col_types = cols())                           
head(survey_response_df)

  |======================================================================| 100%


startDate,endDate,status,progress,duration,finished,recordedDate,_recordId,distributionChannel,userLanguage,⋯,QID72_3,QID72_6,QID72_4,QID73_1,QID73_2,QID73_3,QID73_5,QID74,QID75,QID77_TEXT
<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<lgl>,<dttm>,<chr>,<chr>,<chr>,⋯,<chr>,<lgl>,<lgl>,<chr>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>
2022-04-22 12:28:08,2022-04-22 12:51:57,IP Address,100,1429,TRUE,2022-04-22 12:51:58,R_1FfW01qiIDYgg3j,anonymous,EN,⋯,NA,NA,NA,Very often (5 or more times per month),NA,NA,NA,No,Yes,hawkinsr@nmsu.edu
2022-04-22 14:48:36,2022-04-22 14:55:41,IP Address,100,425,TRUE,2022-04-22 14:55:42,R_2f2TG06j0f54tl2,anonymous,EN,⋯,9 hours or more per night,NA,NA,Very often (5 or more times per month),NA,NA,NA,Prefer not to respond,Yes,kakakari@nmsu.edu


In [ ]:
survey_question_df <- survey_questions(surveys$id[1])
head(survey_question_df, 2)

#### **Cleaning Data Pulled From Quatrics API**

In [ ]:
demography_df = survey_response_df %>%
transmute(DateTime = survey_response_df$recordedDate[1] %>% as.character %>% strsplit(split = " "), Date = DateTime[[1]][1],
Time = DateTime[[1]][2], DateTime = NULL,
Q1 = paste( QID1_1, ' ', QID1_2, ' ', QID1_3, ' ',QID1_4, ' ', QID1_5, ' ', QID1_4_TEXT), Q1 = gsub("NA", "", Q1), Q1 = trimws(gsub("\\s+", " ", Q1)), 
Q2 = QID2,
Q3 = QID3,
Q4 = QID4,
Q5 = paste( QID5_1, ' ', QID5_2, ' ', QID5_3, ' ',QID5_4, ' ', QID5_5, ' ', QID5_6, ' ', QID5_7, ' ', QID5_8, ' ', QID5_7_TEXT), Q5 = gsub("NA", "", Q5), Q5 = trimws(gsub("\\s+", " ", Q5)),
Q6.1 = QID8,
Q6.2 = QID10,
Q6.3 = QID9,
Q6.4 = paste( QID11, ' ', QID11_5_TEXT), Q6.4 = gsub("NA", "", Q6.4), Q6.4 = trimws(gsub("\\s+", " ", Q6.4)), 
Q6.5 = QID15_TEXT,
Q6.6 = QID16,
Q6.7 = QID17,
Q6.8 = paste( QID18_1, ' ', QID18_2, ' ', QID18_3, ' ',QID18_4, ' ', QID18_5, ' ', QID18_6), Q6.8 = gsub("NA", "", Q6.8), Q6.8 = trimws(gsub("\\s+", " ", Q6.8)), 
Q7 = QID14,
Q8 = QID19,
Q9 = QID20,
Q10 = QID21,
Q11 = QID22,
Q12 = QID23,
Q12.1 = paste( QID24_1, ' ', QID24_2, ' ', QID24_3, ' ',QID24_4, ' ', QID24_5, ' ', QID24_6), Q12.1 = gsub("NA", "", Q12.1), Q12.1 = trimws(gsub("\\s+", " ", Q12.1)), 
Q12.2 = QID25,
Q13 = QID27,
Q13.1 = QID28,
Q13.2 = paste( QID29_1, ' ', QID29_2, ' ', QID29_3, ' ',QID29_4, ' ', QID29_5, ' ', QID29_6, ' ', QID29_7), Q13.2 = gsub("NA", "", Q13.2), Q13.2 = trimws(gsub("\\s+", " ", Q13.2)), 
Q13.3 = QID30,
Q13.4 = QID31_TEXT,
Q13.5 = QID32_TEXT,
Q13.6 = QID33,
Q14 = QID34_TEXT,
Q15 = QID35_TEXT,
Q16 = QID36,
Q16.1 = QID37,
Q17 = QID38,
Q18 = QID39,
Q19 = paste( QID40_1, ' ', QID40_2, ' ', QID40_3, ' ',QID40_4, ' ', QID40_5, ' ', QID40_6, ' ', QID40_5_TEXT), Q19 = gsub("NA", "", Q19), Q19 = trimws(gsub("\\s+", " ", Q19)), 
Q20 = paste( QID41_1, ' ', QID41_2, ' ', QID41_3, ' ',QID41_4, ' ', QID41_5, ' ', QID41_6, ' ', QID41_7, ' ', QID41_6_TEXT), Q20 = gsub("NA", "", Q20), Q20 = trimws(gsub("\\s+", " ", Q20)), 
Q21.1 = QID42,
Q21.2 = QID43,
Q21.3 = QID44,
Q21.4 = QID45,
Q21.5 =  paste( QID46, ' ', QID46_7_TEXT), Q21.5 = gsub("NA", "", Q21.5), Q21.5 = trimws(gsub("\\s+", " ", Q21.5)),
Q21.6 = paste( QID47, ' ', QID47_4_TEXT), Q21.6 = gsub("NA", "", Q21.6), Q21.6 = trimws(gsub("\\s+", " ", Q21.6)),
Q21.7 = paste( QID48, ' ', QID47_4_TEXT), Q21.7 = gsub("NA", "", Q21.7), Q21.7 = trimws(gsub("\\s+", " ", Q21.7)),
Q21.8 = paste( QID49, ' ', QID49_1_TEXT), Q21.8 = gsub("NA", "", Q21.8), Q21.8 = trimws(gsub("\\s+", " ", Q21.8)),
Q22 = QID50,
Q22.1 = paste( QID51, ' ', QID51_1_TEXT,' ', QID51_4_TEXT), Q22.1 = gsub("NA", "", Q22.1), Q22.1 = trimws(gsub("\\s+", " ", Q22.1)),
Q22.2 = QID52_TEXT


)

In [ ]:
head(demography_df)

Date,Time,Q1,Q2,Q3,Q4,Q5,Q6.1,Q6.2,Q6.3,⋯,Q21.2,Q21.3,Q21.4,Q21.5,Q21.6,Q21.7,Q21.8,Q22,Q22.1,Q22.2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
2022-04-22,12:51:58,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,No,⋯,No,No,No,No,No,No,No,Yes,,NA
2022-04-22,12:51:58,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,"Yes, I am a student worker",⋯,No,No,No,No,No,No,No,Yes,,NA


In [ ]:
covid_19_df = survey_response_df %>%
transmute(
Q1 = QID53,
Q2 = QID54,
Q3 = QID55,
Q4.1 = QID56,
Q4.2 = QID57,
Q4.3 = QID58,
Q4.4 = QID59,
Q4.5 = QID118,
Q5 = paste( QID60, ' ', QID60_1_TEXT), Q5 = gsub("NA", "", Q5), Q5 = trimws(gsub("\\s+", " ", Q5)), 
Q6 = QID61,
Q6.1 = QID115,
Q7.1 = QID62,
Q7.2 = QID63,
Q7.3 = QID64,
Q7.4 = QID65,
Q7.5 = QID99,
Q8 = paste( QID66, ' ', QID66_1_TEXT), Q5 = gsub("NA", "", Q5), Q5 = trimws(gsub("\\s+", " ", Q5)), 
Q9 = QID67,
Q10 = QID68_TEXT

)

head(covid_19_df)

Q1,Q2,Q3,Q4.1,Q4.2,Q4.3,Q4.4,Q4.5,Q5,Q6,Q6.1,Q7.1,Q7.2,Q7.3,Q7.4,Q7.5,Q8,Q9,Q10
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
No,No,No,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Vaccinated,2 doses,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond NA,Yes,"Work travel, depression"
Yes,Yes,No,No,No,No,No,Yes,"Yes (Please specify [i.e. parent, sibling, etc.]) great grandparent",Vaccinated,3 doses,"Yes, all living parents are vaccinated","Yes, all living siblings are vaccinated",Prefer not to respond,Prefer not to respond,"Yes, all living grandparents are vaccinated",No NA,No,fear of being unable to work due to quarantine but not receive any form of financial compensation


In [ ]:
depresssion_df = survey_response_df %>%
transmute(
Q1 = QID102,
Q2 = QID103,
Q3 = QID104,
Q4 = QID105,
Q5 = QID106,
Q6 = QID107,
Q7 = QID108,
Q8 = QID109,
Q9 = QID110

)

head(depresssion_df)

Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,2,2,3,2,3,3,1,1
2,3,3,3,1,1,3,1,0


In [211]:
sleep_df = survey_response_df %>%
transmute(
Q1 = QID71,
Q2 = paste( QID72_1, ' ', QID72_2, ' ', QID72_3, ' ',QID72_4), Q2 = gsub("NA", "", Q2), Q2 = trimws(gsub("\\s+", " ", Q2)), 
Q3 = paste( QID73_1, ' ', QID73_2, ' ', QID73_3, ' ',QID73_5), Q3 = gsub("NA", "", Q3), Q3 = trimws(gsub("\\s+", " ", Q3)), 
Q4 = QID74,
Compensation = QID75,
Email = QID77_TEXT
)

head(sleep_df)

Q1,Q2,Q3,Q4,Compensation,Email
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month),Prefer not to respond,Yes,kakakari@nmsu.edu


In [ ]:
data_df = cbind(demography_df, covid_19_df, depresssion_df, sleep_df)

In [ ]:
head(data_df)

,Date,Time,Q1,Q2,Q3,Q4,Q5,Q6.1,Q6.2,Q6.3,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2022-04-22,12:51:58,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,No,⋯,3,3,1,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month) NA NA NA,No,Yes,hawkinsr@nmsu.edu
2,2022-04-22,12:51:58,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,"Yes, I am a student worker",⋯,1,3,1,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month) NA NA NA,Prefer not to respond,Yes,kakakari@nmsu.edu


In [ ]:
dim(data_df)

[1]  2 83

#### **Automation Integrity**

In [ ]:
# get file that is used in setting up this automation
suppressMessages({integrity_df = read_excel("Covid_Integrity_Apr_23_5_09_Am.xlsx", col_types = "text")})
head(integrity_df)

22XXX = Undergraduate students,...2,...3,...4,55XXX = Graduate students,...6,...7,77XXX = Faculty,...9,99001 = Staff,⋯,...75,...76,...77,...78,...79,...80,...81,...82,...83,...84
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,I: Demographics,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,IV: Sleep Disorder Questionnaire,NA,NA,NA,NA,NA
Response#,Date,Time,Q1,Q2,Q3,Q4,Q.5,Q6.1,Q6.2,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
22001.0,44673.0,12:52pm,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,⋯,3.0,3.0,1.0,1.0,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
22002.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22003.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22004.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
intergrity_vec = integrity_df[2,] %>% t()
rownames(intergrity_vec) = NULL
intergrity_vec

In [149]:
# get latest file downloaded from one_drive
suppressMessages({one_drive_df = read_excel("Prevalence of Depression and Sleep Disorders in New Mexico State University Populations during COVID-19 - Response Tracker.xlsx", col_types = "text")})
head(one_drive_df)

22XXX = Undergraduate students,...2,...3,...4,55XXX = Graduate students,...6,...7,77XXX = Faculty,...9,99001 = Staff,⋯,...75,...76,...77,...78,...79,...80,...81,...82,...83,...84
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,I: Demographics,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,IV: Sleep Disorder Questionnaire,NA,NA,NA,NA,NA
ResponseNo,Date,Time,Q1,Q2,Q3,Q4,Q.5,Q6.1,Q6.2,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
22001,44673,12:52pm,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,⋯,3,3,1,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
22002,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22003,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22004,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


##### **Filtering Staff and Faculty Data**

In [150]:
# fixing colum names
one_drive_vec = one_drive_df[2,] %>% t()
rownames(one_drive_vec) = NULL
colnames(one_drive_df) = one_drive_vec
head(one_drive_df, 2)

ResponseNo,Date,Time,Q1,Q2,Q3,Q4,Q.5,Q6.1,Q6.2,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,I: Demographics,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,IV: Sleep Disorder Questionnaire,NA,NA,NA,NA,NA
ResponseNo,Date,Time,Q1,Q2,Q3,Q4,Q.5,Q6.1,Q6.2,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
22001,44673,12:52pm,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,⋯,3,3,1,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
22002,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22003,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22004,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [153]:
# filter groups
student_only_df = data_df_name_fixed %>% filter(grepl("^Student$", Q1...3))
head(student_only_df,2)
student_df = data_df_name_fixed %>% filter(grepl("Student", Q1...3))
head(student_df,2)
staff_df = data_df_name_fixed  %>% filter(grepl("Staff", Q1...3) & !grepl("Faculty", Q1...3))
head(staff_df,2)
faculty_df = data_df_name_fixed  %>% filter(grepl("Faculty", Q1...3))
head(faculty_df,2)

Date,Time,Q1...3,Q2...4,Q3...5,Q4...6,Q5...7,Q6.1...8,Q6.2,Q6.3,⋯,Q7...75,Q8...76,Q9...77,Q1...78,Q2...79,Q3...80,Q4...81,Compensation,Email,repair_names
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
2022-04-22,12:51:58,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,No,⋯,3,1,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month) NA NA NA,No,Yes,hawkinsr@nmsu.edu,TRUE


Date,Time,Q1...3,Q2...4,Q3...5,Q4...6,Q5...7,Q6.1...8,Q6.2,Q6.3,⋯,Q7...75,Q8...76,Q9...77,Q1...78,Q2...79,Q3...80,Q4...81,Compensation,Email,repair_names
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
2022-04-22,12:51:58,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,No,⋯,3,1,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month) NA NA NA,No,Yes,hawkinsr@nmsu.edu,TRUE
2022-04-22,12:51:58,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,"Yes, I am a student worker",⋯,3,1,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month) NA NA NA,Prefer not to respond,Yes,kakakari@nmsu.edu,TRUE


Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


Date,Time,Q1...3,Q2...4,Q3...5,Q4...6,Q5...7,Q6.1...8,Q6.2,Q6.3,⋯,Q7...75,Q8...76,Q9...77,Q1...78,Q2...79,Q3...80,Q4...81,Compensation,Email,repair_names
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>


Date,Time,Q1...3,Q2...4,Q3...5,Q4...6,Q5...7,Q6.1...8,Q6.2,Q6.3,⋯,Q7...75,Q8...76,Q9...77,Q1...78,Q2...79,Q3...80,Q4...81,Compensation,Email,repair_names
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
2022-04-22,12:51:58,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,"Yes, I am a student worker",⋯,3,1,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month) NA NA NA,Prefer not to respond,Yes,kakakari@nmsu.edu,TRUE


In [151]:
suppressMessages({one_drive_df_name_fixed = tibble(one_drive_df, repair_names = TRUE, .name_repair = "universal")})

In [152]:
suppressMessages({data_df_name_fixed = tibble(data_df, repair_names = TRUE, .name_repair = "universal")})

##### **Inserting Response Number**

In [178]:
faculty_df = faculty_df %>% mutate(ResponseNo = 77001:(77000 + n()) ) %>% dplyr::select(ResponseNo, everything())
head(faculty_df)

staff_df = tryCatch(
{
staff_df %>% mutate(ResponseNo = 99001:(99000 + n()) ) %>% dplyr::select(ResponseNo, everything())
},
error = function(e){
        rbind(staff_df, rep("", ncol(staff_df))) %>% mutate(ResponseNo = 99001:(99000 + n()) ) %>% dplyr::select(ResponseNo, everything())
    }
)
head(staff_df)





ResponseNo,Date,Time,Q1...3,Q2...4,Q3...5,Q4...6,Q5...7,Q6.1...8,Q6.2,⋯,Q7...75,Q8...76,Q9...77,Q1...78,Q2...79,Q3...80,Q4...81,Compensation,Email,repair_names
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
77001,2022-04-22,12:51:58,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,⋯,3,1,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month) NA NA NA,Prefer not to respond,Yes,kakakari@nmsu.edu,TRUE


,ResponseNo,X..,X...1,X...2,X...3,X...4,X...5,X...6,X...7,X...8,⋯,X...74,X...75,X...76,X...77,X...78,X...79,X...80,X...81,X...82,X...83
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,99001,,,,,,,,,,⋯,,,,,,,,,,


###### **Find Where to Start Writing**

In [175]:
one_drive_df_name_fixed$Row_num = 2:(nrow(one_drive_df_name_fixed) + 1)
faculty_start_write = one_drive_df_name_fixed %>% filter(ResponseNo == 77001) %>% select(Row_num)
staff_start_write = one_drive_df_name_fixed %>% filter(ResponseNo == 99001) %>% select(Row_num)
faculty_start_write
staff_start_write

Row_num
<int>
204


Row_num
<int>
304


##### **Writing Data To Workbook**

In [194]:
# Write Dataframe
wb <- loadWorkbook("Prevalence of Depression and Sleep Disorders in New Mexico State University Populations during COVID-19 - Response Tracker.xlsx")

writeData(wb, sheet = "Sheet1", x = staff_df, startRow = staff_start_write, rowNames = FALSE, colNames = FALSE)

writeData(wb, sheet = "Sheet1", x = faculty_df, startRow = faculty_start_write, rowNames = FALSE, colNames = FALSE)
border_col_num = c(3, 50, 69, 78, 82, 84)
# style 
s1 <- createStyle(fontSize = 11, wrapText = TRUE, halign = "center")
s2 <-  createStyle(border = "right")

# # Apply styles to the workbook
addStyle(wb, sheet = "Sheet1", style = s1, stack = TRUE,  rows = 1:(nrow(one_drive_df_name_fixed)+1), cols = 1:(ncol(one_drive_df_name_fixed)+1), gridExpand = TRUE)
addStyle(wb, sheet = "Sheet1", style = s2, stack = TRUE,  rows = 1:(nrow(one_drive_df_name_fixed)+1), cols = border_col_num, gridExpand = TRUE)

saveWorkbook(wb, paste0(Sys.Date()," Test_file (openxlsx)",".xlsx"), overwrite = TRUE)

In [204]:
# get_personal_onedrive(auth_type="device_code")

# get_business_onedrive(tenant="nmsu.edu", auth_type="device_code")

Creating Microsoft Graph login for tenant 'nmsu.edu'



To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E44A7KYSH to authenticate. 


Waiting for device code in browser...
Press Esc/Ctrl + C to abort



In [212]:
# read_url_csv <- function(url, ...){
#   tmpFile <- tempfile()
#   download.file(url, destfile = tmpFile)
#   url_csv <- readr::read_csv(tmpFile, ...)
#   return(url_csv)
# }
# onedrive_url <- "https://eltnmsu-my.sharepoint.com/:x:/g/personal/jonaggie_nmsu_edu/ETrw5EUUJW1OkfoqgknUWSMB9mgomEJ6xf7qyite3k4U5A"

# csv <- read_url_csv(onedrive_url)

